# MCP

This tutorial will demonstrate how to integrate [Model Context Protocol (MCP)](https://modelcontextprotocol.io/) with Bridgic to enhance your development in building agentic applications.

## Introduction

**Model Context Protocol (MCP)** is an open protocol that enables AI applications to securely access external resources and tools. By integrating MCP with Bridgic, you can:

- **Connect to MCP Servers**: Access a wide range of external services and resources through standardized MCP servers
- **Get and Use MCP Tools**: Leverage tools provided by MCP servers as workers in your Bridgic workflows
- **Get and Use MCP Prompts**: Utilize pre-configured prompt templates from MCP servers
- **Build Intelligent Agents**: Enable LLM-driven agents to autonomously select and use MCP tools

This tutorial will walk you through the essentials of integrating MCP with Bridgic, from basic installation to advanced usage, along with easy-to-understand examples.


## Installation

First, install the `bridgic-protocols-mcp` package. Since the MCP Python SDK requires Python 3.12 or newer. Please ensure you are using a compatible Python version before installation.

```shell
pip install bridgic-protocols-mcp
```

Let's verify the installation by running:

```shell
python -c "from bridgic.protocols.mcp import __version__; print(__version__)"
```


## Basic Usage

### Connecting to an MCP Server

MCP servers can be connected via different transport protocols. The most common is **stdio** transport, which runs the MCP server as a subprocess. Let's connect to a filesystem MCP server as an example:


In [1]:
import os
import tempfile

from bridgic.protocols.mcp import McpServerConnectionStdio

# Create a temporary directory for the filesystem MCP server
temp_dir = os.path.realpath(tempfile.mkdtemp())
print(f"Using temporary directory: {temp_dir}")

# Create a connection to a filesystem MCP server
# Note: This requires Node.js and npx to be installed
connection = McpServerConnectionStdio(
    name="connection-filesystem-stdio",
    command="npx",
    args=["-y", "@modelcontextprotocol/server-filesystem", temp_dir],
)

# Establish the connection
connection.connect()

# Verify connection
print(f"✓ Connected to MCP server: {connection.name}")
print(f"  Connection status: {connection.is_connected}")

# List available tools
tools = connection.list_tools()
print(f"\n✓ Found {len(tools)} available tools:")
for tool in tools:
    print(f"  - {tool.tool_name}: {tool.tool_description}")

Using temporary directory: /private/var/folders/9t/5r9fms9s5q33p6xty_0_k1mw0000gn/T/tmp5wlu35c8
✓ Connected to MCP server: connection-filesystem-stdio
  Connection status: True

✓ Found 14 available tools:
  - read_file: Read the complete contents of a file as text. DEPRECATED: Use read_text_file instead.
  - read_text_file: Read the complete contents of a file from the file system as text. Handles various text encodings and provides detailed error messages if the file cannot be read. Use this tool when you need to examine the contents of a single file. Use the 'head' parameter to read only the first N lines of a file, or the 'tail' parameter to read only the last N lines of a file. Operates on the file as text regardless of extension. Only works within allowed directories.
  - read_media_file: Read an image or audio file. Returns the base64 encoded data and MIME type. Only works within allowed directories.
  - read_multiple_files: Read the contents of multiple files simultaneously. Th

### Using a MCP Tool as a Worker

MCP tools can be converted to Bridgic workers and integrated into `GraphAutoma` building. This allows you to orchestrate MCP tool calls alongside other workers in your application.

Why don't we just run the tool directly, but instead execute it as a worker? In Bridgic's view, every execution process in a workflow program (or an even more agentic system) can be decomposed into fine-grained workers, which can then be orchestrated and scheduled. Standardizing the execution process in this way simplifies development and debugging, and enhances observability during execution. Currently, too many frameworks separate agent operation from programmable orchestration, causing tools and developer-defined work units to hold unequal positions. This leads to two very different development and debugging experiences.

In the world of Bridgic, a tool is a worker: both are work units with clearly defined inputs and expected outputs, which are connected together, either due to predefined conditions or runtime ones, to accomplish more complex and bigger tasks.

Let's create a simple workflow that uses MCP tools to read and write files:


In [3]:
import datetime
import mcp

from bridgic.core.automa import GraphAutoma, RunningOptions, worker
from bridgic.core.automa.args import System

# List the tools via the server connection
tools = connection.list_tools()

# Filter the needed one which will create the real worker
write_tool = next(t for t in tools if t.tool_name == "write_file")
read_tool = next(t for t in tools if t.tool_name == "read_file")
meta_tool = next(t for t in tools if t.tool_name == "get_file_info")

class FileWriter(GraphAutoma):
    def __init__(self, name: str, running_options: RunningOptions = None):
        super().__init__(name=name, running_options=running_options)
        self.add_worker("write", write_tool.create_worker())
        self.add_worker("read", read_tool.create_worker())
        self.add_worker("meta", meta_tool.create_worker())

    @worker(is_start=True)
    def start(self, title: str, content: str, rtx = System("runtime_context")):
        # Get the current time
        now_time = datetime.datetime.now()

        # Get the content and path of the file to be written
        file_path = f"{temp_dir}/{title}.txt"
        file_content = (
            f"Time: {now_time.strftime('%Y-%m-%d %H:%M:%S')}\n"
            f"Content: {content}\n"
        )

        # Write the file at the next step
        self.ferry_to("write", content=file_content, path=file_path)

        return file_path

    @worker(dependencies=["start", "write"])
    def after_write(self, file_path: str, write_info: mcp.types.CallToolResult):
        self.ferry_to("read", path=file_path)
        self.ferry_to("meta", path=file_path)

    @worker(is_output=True, dependencies=["start", "read", "meta"])
    def output(self, file_path: str, read_info: mcp.types.CallToolResult, meta_info: mcp.types.CallToolResult) -> str:
        return (
            f"✓ Finnished writting!"
            f"\nFile path: {file_path}"
            f"\n{meta_info.content[0].text}"
        )

file_processor = FileWriter(name="file-processor")

for content in [
    ("1.txt", "Hello, Bridgic!"),
    ("2.txt", "Hello, MCP!"),
]:
    result = await file_processor.arun(title=content[0], content=content[1])
    print(f"\n{result}")


✓ Finnished writting!
File path: /private/var/folders/9t/5r9fms9s5q33p6xty_0_k1mw0000gn/T/tmp5wlu35c8/1.txt.txt
size: 51
created: Wed Jan 21 2026 18:18:03 GMT+0800 (China Standard Time)
modified: Wed Jan 21 2026 18:18:03 GMT+0800 (China Standard Time)
accessed: Wed Jan 21 2026 18:18:03 GMT+0800 (China Standard Time)
isDirectory: false
isFile: true
permissions: 644

✓ Finnished writting!
File path: /private/var/folders/9t/5r9fms9s5q33p6xty_0_k1mw0000gn/T/tmp5wlu35c8/2.txt.txt
size: 47
created: Wed Jan 21 2026 18:18:03 GMT+0800 (China Standard Time)
modified: Wed Jan 21 2026 18:18:03 GMT+0800 (China Standard Time)
accessed: Wed Jan 21 2026 18:18:03 GMT+0800 (China Standard Time)
isDirectory: false
isFile: true
permissions: 644
